In [1]:
import os
import json
import glob

In [2]:
# Function to restructure comments
def restructure_comments(data):
    restructured = {
        "tid": data["tid"],
        "name": data["name"],
        "author": data["author"],
        "author_id": data["author_id"],
        "story": data["story"],
        "likeCount": data["likeCount"],
        "emoCount": data["emoCount"],
        "emotions": data["emotions"],
        "tagList": data["tagList"],
        "dateTime": data["dateTime"],
        "commentCount": data["commentCount"],
        "comments": []
    }

    # Helper function to filter emotions
    def filter_emotions(emotions):
        return {k: v for k, v in emotions.items() if v != 0}

    # Traverse through comments to create structured conversation
    for comment in data["comments"]:
        comment_emotions = filter_emotions(comment["emotions"])
        comment_entry = {
            "comment": comment["message"],
            "comment_author": comment["user_name"],
            "comment_dateTime": comment["dateTime"],
            "replies": []
        }
        if comment_emotions:
            comment_entry["comment_emotions"] = comment_emotions

        if comment["replyCount"] > 0:
            for reply in comment["replies"]:
                reply_emotions = filter_emotions(reply["emotions"])
                reply_entry = {
                    "reply": reply["message"],
                    "reply_author": reply["user_name"],
                    "reply_dateTime": reply["dateTime"]
                }
                if reply_emotions:
                    reply_entry["reply_emotions"] = reply_emotions
                comment_entry["replies"].append(reply_entry)
        restructured["comments"].append(comment_entry)
    
    return restructured

# Function to process all files in a folder
def process_files(input_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get all JSON files in the input folder
    json_files = glob.glob(os.path.join(input_folder, '*.json'))

    for file_path in json_files:
        # Read data from file
        with open(file_path, 'r', encoding='utf-8-sig') as f:
            data = json.load(f)

        # Restructure the comments
        restructured_data = restructure_comments(data)

        # Get the filename and create the new file path
        filename = os.path.basename(file_path)
        new_file_path = os.path.join(output_folder, filename)

        # Save the restructured data to a new JSON file
        with open(new_file_path, 'w', encoding='utf-8') as f:
            json.dump(restructured_data, f, ensure_ascii=False, indent=2)

# Define the input and output folders
input_folder = 'pantip_storage'
output_folder = 'restructured_data'

# Process all files
process_files(input_folder, output_folder)
